In [1]:
using QuantEcon, Interpolations, Plots

In [2]:
struct Parameters
    α_1::Float64
    α_2::Float64
    γ_h::Float64
    intercept::Float64
    age1::Float64
    age2::Float64
    age3::Float64
    child::Float64
    interact::Float64
    ρ::Float64
    σ::Float64
    gp_ν_s::Int64
    grid_ν_s::Vector{Float64}
    markov::Matrix{Float64}
    aux_ages::Array{Int64}
    aux_ν::Array{Float64}
    #int_ν_j = interpolate(knots, aux_ν, Gridded(Linear()))
    #min_h::Float64
    #max_h::Float64
    gp_h::Int64
    grid_h::Vector{Float64}
    J::Int64
    β::Float64

    function Parameters(α_1 = 0.351;
                        α_2 = 0.379,
                        γ_h = 1.0,
                        intercept = 0.196,
                        age1 = 3.32,
                        age2 = -0.086,
                        age3 = 0.0007,
                        child = -6.81,
                        interact = 0.133,
                        ρ = 0.76,
                        σ = 0.79,
                        ν_s = 1,
                        gp_ν_s = 7,
                        aux_ages = [17, 20, 25, 30, 40, 50, 55, 60, 65],
                        aux_ν = [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6],
                        min_h = 0.0001,
                        max_h = 200,
                        gp_h = 101,
                        J = 4*(65-20),
                        β = 0.99)
        grid_h = collect(range(min_h, stop = max_h, length=gp_h))
        m = QuantEcon.tauchen(gp_ν_s,ρ,σ,0,1)
        markov = m.p
        grid_ν_s = exp.(collect(m.state_values))
        new(α_1, α_2, γ_h, intercept, age1, age2, age3, child, interact, ρ, σ, gp_ν_s, grid_ν_s, markov, aux_ages, aux_ν,  gp_h, grid_h, J, β)
    end
end

In [3]:
function c(j,h,P::Parameters)
    return (P.α_1+j^(P.α_2))*h^(P.γ_h)
end
function HCA(c,V_hat,V_hat_delta)
    e = max(1-c/(V_hat_delta-V_hat),0) #closed form solution of optimal effort
    if e>1
        error("outside of definition")
    end
    return (c*log(1-e)+e*V_hat+(1-e)*V_hat_delta, e)
end

function l(j,n,P::Parameters)
    P.intercept+P.age1*j+P.age2*j^2+P.age3*j^3+P.child*n+P.interact*n*j
end

function W(j,h,n,ν,VV_hat,VV_hat_delta,ll,P::Parameters)
    cc = c(j,h,P)
    (opt_HCA,e) = HCA(cc,VV_hat,VV_hat_delta)
    uu = h*ν
    aux_val = h*ll + (1-ll)*uu + P.γ_h*log(1+n) + opt_HCA
    return (aux_val, e)
end

function H(j,h,n,ν,γ_n,VV_hat)
    uu = h*ν
    return uu + γ_n*log(1+n)+VV_hat
end

function V(j,h,n,P::Parameters)
    max(H(j,h,n,P::Parameters),V(j,h,n,P::Parameters))
end

V (generic function with 1 method)

In [4]:
function male_policy(P::Parameters)
    W_value = Array{Float64}(undef, P.gp_h, P.J, P.gp_ν_s)
    H_value = Array{Float64}(undef, P.gp_h, P.J, P.gp_ν_s)
    V_value = Array{Float64}(undef, P.gp_h, P.J, P.gp_ν_s)
    labor_policy = Array{Float64}(undef, P.gp_h, P.J, P.gp_ν_s)
    effort_policy = Array{Float64}(undef, P.gp_h, P.J, P.gp_ν_s)
    knots = (P.aux_ages,)
    int_ν_j = interpolate(knots, P.aux_ν, Gridded(Linear()))

    #Last period
    ll = l(P.J,0,P)
    for (ind_h,h) in enumerate(P.grid_h)
        age = 20+P.J/4
        for (ind_ν, ν_s) in enumerate(P.grid_ν_s)
            ν = int_ν_j[age]*ν_s
            (W_value[ind_h,P.J,ind_ν], effort_policy[ind_h, P.J,ind_ν]) = W(P.J,h,0,ν,10^(-8),10^(-6),ll,P)
            H_value[ind_h,P.J,ind_ν]                                  = H(P.J,h,0,ν,P.γ_h,10^(-8))
            V_value[ind_h,P.J,ind_ν]                                  = max.(W_value[ind_h,P.J,ind_ν],H_value[ind_h,P.J,ind_ν])
            labor_policy[ind_h,P.J,ind_ν]                             = 1*(W_value[ind_h,P.J,ind_ν]>H_value[ind_h,P.J,ind_ν])
        end
    end
    #Backwards induction
    for j in P.J-1:-1:1
        #j=J-1
        age = 20+j/4
        ll = l(j,0,P)
        #println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s),V_value[:,j+1,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        for (ind_h,h) in enumerate(P.grid_h)
            h_delta = h*1.03
            for (ind_ν, ν_s) in enumerate(P.grid_ν_s)
                contval = 0
                contval_delta = 0
                for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                    contval       = contval      + P.markov[ind_ν,ind_ν_2]*etp_h[h,ν_s_2]
                    contval_delta = contval_delta+ P.markov[ind_ν,ind_ν_2]*etp_h[h_delta,ν_s_2]
                end
                ν = int_ν_j[age]*ν_s
                (W_value[ind_h,j,ind_ν], effort_policy[ind_h, j,ind_ν]) = W(j, h, 0, ν, contval, contval_delta, ll, P)
                H_value[ind_h,j,ind_ν] = H(j,h,0,ν,P.γ_h, contval)
                V_value[ind_h,j,ind_ν] = max.(W_value[ind_h,j,ind_ν],H_value[ind_h,j,ind_ν])
                labor_policy[ind_h,j,ind_ν] = 1*(W_value[ind_h,j,ind_ν]>H_value[ind_h,j,ind_ν])
            end
        end
    end
    return (W_value, H_value, V_value, labor_policy, effort_policy)
end

male_policy (generic function with 1 method)

In [ ]:
P = Parameters()

In [ ]:
(W_value, H_value, V_value, labor_policy, effort_policy) = male_policy(P)